In [25]:
import numpy as np
from PIL import Image
from sklearn.feature_extraction.image import extract_patches_2d, reconstruct_from_patches_2d
from sklearn.preprocessing import normalize

from dictlearn import DictionaryLearning
from dictlearn import methods

In [13]:
p = 8
s = 6
N = 1000
n = 256
K = 50
sigma = 0.075


In [35]:
img = Image.open("barbara.png")
I = np.asarray(img)
Inoisy = I + sigma * np.random.randn(I.shape[0], I.shape[1])

Ynoisy = extract_patches_2d(Inoisy, (p, p))

print(Ynoisy.size)
print(Ynoisy.shape)

Ynoisy = Ynoisy.reshape(Ynoisy.shape[0], -1)

print(Ynoisy.size)
print(Ynoisy.shape)

Ynoisy = Ynoisy.T

mean = np.mean(Ynoisy, 0)
Ynoisy = Ynoisy - mean

idx = np.random.choice(Ynoisy.shape[1], N)
Y = Ynoisy[:, idx]
print(Y)

16321600
(255025, 8, 8)
16321600
(255025, 64)
[[ 18.83190942  11.1812841    2.6625968  ...  69.55610024  71.21428048
  -10.64184112]
 [ 19.96460917   8.95150203   0.71238085 ...  78.50283393  33.36024549
  -10.59118501]
 [ 16.99795189  10.07766851   0.76710832 ...  80.7109107   -9.74914588
   -6.75411412]
 ...
 [-23.10571115  -3.96237018   5.70774748 ... -59.44826135 -93.76189293
    8.35065739]
 [-23.18193173 -20.84668087  -1.3239983  ... -57.53571396 -95.81938975
    6.42313581]
 [-22.07600775 -65.98044885   3.66492414 ... -54.45453832 -95.78728061
   10.29107853]]


In [36]:
D0 = np.random.randn(p * p, N)
D0 = normalize(D0, axis=0, norm='max')

dl = DictionaryLearning(
    n_components=n,
    max_iter=K,
    fit_algorithm='ksvd',
    n_nonzero_coefs=s,
    code_init=None,
    dict_init=D0,
    params=None,
    data_sklearn_compat=False
)
dl.fit(Y)
D = dl.D_

In [38]:
Xc, err = methods.omp(Ynoisy, D, n_nonzero_coefs=s)

In [46]:
Yc = np.matmul(D, Xc) + mean
print(Yc.shape)

(64, 255025)


In [53]:
Yc = Yc.reshape((8, 8, Yc.shape[1]))

ValueError: cannot reshape array of size 16321600 into shape (8,8,8)

In [57]:
Yc = np.transpose(Yc, (2, 0, 1))

In [58]:
Ic = reconstruct_from_patches_2d(Yc, (I.shape[0], I.shape[1]))

In [60]:
img.show()
Image.fromarray(Inoisy).show()
Image.fromarray(Ic).show()

NameError: name 'img' is not defined

In [61]:
def psnr(img1, img2):
    mse = np.mean((img1 - img2) ** 2)
    if mse == 0:
        return 0
    max_pixel = 255
    res = 20 * np.log10(max_pixel / np.sqrt(mse))
    return res


psnr1 = psnr(I, Ic)
psnr2 = psnr(I, Inoisy)

print(psnr1, psnr2)

37.71135514324791 70.65054974519543
